In [1]:
#handling data
import pandas as pd
import numpy as np
from scipy import stats
from operator import itemgetter


In [2]:
# Read json into a pandas dataframe
tweets_df = pd.read_json("insurance.txt", lines=True)

In [3]:
tweets_df.columns

Index(['created_at', 'id', 'id_str', 'text', 'source', 'truncated',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'is_quote_status', 'extended_tweet', 'quote_count',
       'reply_count', 'retweet_count', 'favorite_count', 'entities',
       'favorited', 'retweeted', 'possibly_sensitive', 'filter_level', 'lang',
       'timestamp_ms', 'display_text_range', 'retweeted_status',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status',
       'quoted_status_permalink', 'extended_entities'],
      dtype='object')

In [4]:
all_hashtags = []
for e in tweets_df.entities:
    hashtags = [t['text'] for t in e['hashtags']]
    all_hashtags = all_hashtags + hashtags

from collections import Counter
c = Counter(all_hashtags)
most_common_tuples = c.most_common()
sorted_keys = sorted(c, key=c.get, reverse=True)
most_common_tuples

[('fintech', 256),
 ('AI', 175),
 ('insurance', 101),
 ('BigData', 67),
 ('IoT', 63),
 ('Fintech', 58),
 ('Insurance', 49),
 ('insurtech', 43),
 ('legal', 43),
 ('COVID19', 42),
 ('finserv', 39),
 ('ArtificialIntelligence', 38),
 ('FinTech', 36),
 ('MachineLearning', 34),
 ('DataScience', 33),
 ('Blockchain', 31),
 ('blockchain', 28),
 ('digital', 28),
 ('goldenmoneyteam', 28),
 ('CyberSecurity', 27),
 ('agodgmt', 27),
 ('banking', 26),
 ('Analytics', 26),
 ('innovation', 25),
 ('DigitalTransformation', 24),
 ('Bitcoin', 23),
 ('ai', 23),
 ('DataScientist', 22),
 ('MarketScreener', 21),
 ('payments', 20),
 ('law', 20),
 ('Insurtech', 19),
 ('startup', 19),
 ('DeepLearning', 19),
 ('robot', 18),
 ('Goldenmoneyteam', 18),
 ('Spotify', 18),
 ('EmergingTechnologies', 15),
 ('Pandemic', 15),
 ('crypto', 14),
 ('Innovation', 14),
 ('HealthTech', 13),
 ('cryptocurrency', 13),
 ('robotics', 13),
 ('ML', 12),
 ('regtech', 12),
 ('claim', 12),
 ('VR', 12),
 ('insurers', 11),
 ('finance', 11),
 (

In [6]:
hashtag_dict = {}
for t in most_common_tuples:
    hashtag_dict[t[0]] = t[1]

In [15]:
hashtag_wordclouds = []
for t in most_common_tuples:
    hashtag_wordclouds.append({'text': t[0], 'value': t[1]})

In [7]:
hashtag_dict

{'fintech': 256,
 'AI': 175,
 'insurance': 101,
 'BigData': 67,
 'IoT': 63,
 'Fintech': 58,
 'Insurance': 49,
 'insurtech': 43,
 'legal': 43,
 'COVID19': 42,
 'finserv': 39,
 'ArtificialIntelligence': 38,
 'FinTech': 36,
 'MachineLearning': 34,
 'DataScience': 33,
 'Blockchain': 31,
 'blockchain': 28,
 'digital': 28,
 'goldenmoneyteam': 28,
 'CyberSecurity': 27,
 'agodgmt': 27,
 'banking': 26,
 'Analytics': 26,
 'innovation': 25,
 'DigitalTransformation': 24,
 'Bitcoin': 23,
 'ai': 23,
 'DataScientist': 22,
 'MarketScreener': 21,
 'payments': 20,
 'law': 20,
 'Insurtech': 19,
 'startup': 19,
 'DeepLearning': 19,
 'robot': 18,
 'Goldenmoneyteam': 18,
 'Spotify': 18,
 'EmergingTechnologies': 15,
 'Pandemic': 15,
 'crypto': 14,
 'Innovation': 14,
 'HealthTech': 13,
 'cryptocurrency': 13,
 'robotics': 13,
 'ML': 12,
 'regtech': 12,
 'claim': 12,
 'VR': 12,
 'insurers': 11,
 'finance': 11,
 'AR': 11,
 'technology': 11,
 'Cloud': 11,
 'lawyers': 11,
 'Legal': 11,
 'DL': 11,
 'HealthInsurance

In [16]:
import json

with open('hashtags.json', 'w') as outfile:
    json.dump(hashtag_dict, outfile)
    

with open('hashtags_wordcloud.json', 'w') as outfile:
    json.dump(hashtag_wordclouds, outfile)

In [10]:
!head hashtags.json

{"fintech": 256, "AI": 175, "insurance": 101, "BigData": 67, "IoT": 63, "Fintech": 58, "Insurance": 49, "insurtech": 43, "legal": 43, "COVID19": 42, "finserv": 39, "ArtificialIntelligence": 38, "FinTech": 36, "MachineLearning": 34, "DataScience": 33, "Blockchain": 31, "blockchain": 28, "digital": 28, "goldenmoneyteam": 28, "CyberSecurity": 27, "agodgmt": 27, "banking": 26, "Analytics": 26, "innovation": 25, "DigitalTransformation": 24, "Bitcoin": 23, "ai": 23, "DataScientist": 22, "MarketScreener": 21, "payments": 20, "law": 20, "Insurtech": 19, "startup": 19, "DeepLearning": 19, "robot": 18, "Goldenmoneyteam": 18, "Spotify": 18, "EmergingTechnologies": 15, "Pandemic": 15, "crypto": 14, "Innovation": 14, "HealthTech": 13, "cryptocurrency": 13, "robotics": 13, "ML": 12, "regtech": 12, "claim": 12, "VR": 12, "insurers": 11, "finance": 11, "AR": 11, "technology": 11, "Cloud": 11, "lawyers": 11, "Legal": 11, "DL": 11, "HealthInsurance": 11, "Insurers": 10, "covid19": 10, "marketing": 10, "

In [19]:
for t in most_common_tuples:
    tweets_df['#'+t[0]] = False

In [20]:
# tweets_df.at[0, '#AppleMusic'] = False

In [21]:
for i, e in enumerate(tweets_df.entities):
    hashtags = [t['text'] for t in e['hashtags']]
    for hashtag in hashtags:
        tweets_df.at[i, '#' + hashtag] = True
    

In [22]:
for t in tweets_df[tweets_df['#insurance'] == True]['text'].values:
    print(t)

Here's a list of common reasons that a #homeowners #insurance policy can be cancelled.  https://t.co/pVITkFIpPV https://t.co/fbsEIdlFEf
RT @PLAYER1Rising: Pretend everyone was rich @SpeakerPelosi or had an #insurance policy protecting them retroactively during a national cri…
RT @floodtweeter: Even if your property is not in a high-risk #flood area, you should consider flood #insurance. Learn more about how #FEMA…
RT @globaliqx: Why data and #AI will become the new normal in #insurance

https://t.co/rBOFO3bho9 

#BigData  #insurtech 

@labordeolivier…
RT @globaliqx: Why data and #AI will become the new normal in #insurance

https://t.co/rBOFO3bho9 

#BigData  #insurtech 

@labordeolivier…
Our #Chiropractors work with many #insurance companies. Do you know whether you are covered? https://t.co/1MpS1krOc3
RT @ESlaw_London: #covid19 and upcoming thought leadership ...
#financialservices #banking #insurance #Assetmanagement #payments #PrivateEq…
Did you know the cost of your #homeowners #i

# Getting media

In [23]:
media_list = list(filter(lambda x: isinstance(x, dict), tweets_df['extended_entities'].values.tolist()))
for media in media_list:
    for m in media['media']:
        print(m['type'], m['media_url'])

photo http://pbs.twimg.com/media/EYAfLwdX0AYvcOG.jpg
photo http://pbs.twimg.com/media/EYAfXCsXgAg_SZr.jpg
photo http://pbs.twimg.com/media/EYAmZrxXsAUV-2e.jpg
photo http://pbs.twimg.com/media/EYAma8wWsAAQQP1.jpg
photo http://pbs.twimg.com/media/EYAohJsWkAAuR3c.jpg
photo http://pbs.twimg.com/media/EYAo0FIX0AMQkNT.jpg
photo http://pbs.twimg.com/media/EX-wCr4WkAgl-8D.jpg
photo http://pbs.twimg.com/media/EYAtL5iXQAA1TOQ.jpg
photo http://pbs.twimg.com/media/EYAuVjQWsAIKmjd.jpg
video http://pbs.twimg.com/ext_tw_video_thumb/1251037581908860931/pu/img/OsaUJKWsK6Glo1Qx.jpg
photo http://pbs.twimg.com/media/EYAwsLMWAAEQwJ8.jpg
video http://pbs.twimg.com/ext_tw_video_thumb/1260946764548489220/pu/img/cXocYxCzWBzes3vX.jpg
photo http://pbs.twimg.com/media/EX-60FDVcA0aSGz.jpg
photo http://pbs.twimg.com/media/EYAypKLWsAE-kxE.jpg
photo http://pbs.twimg.com/media/EYA3zsxU0AAOBGm.jpg
photo http://pbs.twimg.com/media/EYA5NWjVcAcjVPP.jpg
photo http://pbs.twimg.com/media/EYA-PXiXYAE8pX8.jpg
photo http://pbs.

# Full text

In [24]:
import math
def apply_func(x):
    if not isinstance(x, float):
        if 'full_text' in x: 
            return x['full_text']
        else:
            return float('nan')
    else:
        return float('nan')
    

tweets_df['full_text'] = tweets_df['extended_tweet'].apply(lambda x: apply_func(x))
tweets_df.full_text.fillna(tweets_df.text, inplace=True)

In [26]:
!pip install favicon

# Links

In [18]:
import re 
from collections import Counter
import requests

import extraction
import requests

import favicon



url_list = []
for text in tweets_df['text'].values.tolist():
    urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
    url_list += urls

    

url_list = list(filter(lambda url: len(url)>13, url_list))

c = Counter(url_list)
most_common_urls = c.most_common()
sorted_urls = sorted(c, key=c.get, reverse=True)

news_articles = []


for i, url in enumerate(sorted_urls):
    if i == 10:
        break
    resp = requests.head(url)
    try:
        news_article_dict = {}
        
        url = resp.headers["Location"]
        html = requests.get(url).text
        extracted = extraction.Extractor().extract(html, source_url=url)
        icon_url = favicon.get(url)[0][0]
        
        print('title:', extracted.title)
        print('description:', extracted.description)
        print(extracted.image, url, icon_url)
        
        news_article_dict['title'] = extracted.title
        news_article_dict['description'] = extracted.description
        news_article_dict['favicon'] = icon_url
        news_article_dict['image'] = extracted.image
        news_article_dict['url'] = url
        news_article_dict['share_count'] = most_common_urls[i][1]
        
        news_articles.append(news_article_dict)
        
        
        
        
        
        print('\n')
    except:
        print(url)
        print('\n')



title: urbebank.com steht zum Verkauf - Sedo GmbH
description: Keine Daten verfügbar!
https://cdn.sedo.com/images/visa.png http://urbebank.com/ https://cdn.sedo.com/fileadmin_git/resources/public/images/apple-touch-icon.png?v=dLJ3bx2xjj


title: Why data and machine intelligence will become the new normal in insurance | Swiss Re
description: Machine learning and artificial intelligence are the latest tools that purport to improve this pre-emptive ability and make better decisions.
https://www.swissre.com/dam/jcr:aa95fb32-db21-430b-89d2-72b640be5fc0/data-and-machine-intelligencel-in-insurance.jpg https://buff.ly/3bmZxAI https://www.swissre.com/.resources/swissre-web/webresources/img/graphics/favicon-194x194.png


title: Four Artificial Intelligence Technologies to Lead the Global Economy Out of the Pandemic
description: Technology innovation in artificial intelligence (AI) is accelerating at a breakneck pace, and the ability to innovate, adopt and integrate AI techniques to evolve busin

In [17]:
# most_common_urls

[('https://t.co/oD1YCV2EgH', 15),
 ('https://t.co/rBOFO3bho9', 14),
 ('https://t.co/sgxVwESPGQ', 14),
 ('https://t.co/xJgbsFTQXm', 13),
 ('https://t.co/QyIgHoxzdj', 9),
 ('https://t.co/SfQY7NGaKo', 7),
 ('https://t.co/4sdz6nF3xo', 7),
 ('https://t.co/DKZQtIYZVu', 6),
 ('https://t.co/ZJ8QPywYU0', 6),
 ('https://t.co/eGXtMqOorm', 6),
 ('https://t.co/TeL80xqUfK', 6),
 ('https://t.co/5Eq6zerC1Y', 5),
 ('https://t.co/rLdfwpTAKf', 5),
 ('https://t.co/eUlTTW3kiC', 5),
 ('https://t.co/mCC2vadlQo', 4),
 ('https://t.co/lccuXi01u4', 4),
 ('https://t.co/xmMhCtuLU5', 4),
 ('https://t.co/3NKgw00W6J', 4),
 ('https://t.co/Yd1zxcM76n', 4),
 ('https://t.co/WbHUS', 4),
 ('https://t.co/5igXtLMt0Y', 4),
 ('https://t.co/eM7W0pn9Hf', 4),
 ('https://t.co/G163plZR27', 4),
 ('https://t.co/szpOjPqLc8', 4),
 ('https://t.co/ws9mIxl2qc', 4),
 ('https://t.co/Ch0Ev4IE86', 3),
 ('https://t.co/GDnmvL5791', 3),
 ('https://t.co/ZvzOYUXZsy', 3),
 ('https://t.co/PiF8MTgUtY', 3),
 ('https://t.co/T3XZRpw5PZ', 3),
 ('https://

In [19]:
import json

with open('articles.json', 'w') as outfile:
    json.dump(news_articles, outfile)

Apache Kafka Installation on Mac using Homebrew
Hi all,
https://miro.medium.com/max/612/1*tsXkwb5Q57qUzUhC07ESFQ.png
